## Restaurant Recomendation System

### Importing Python Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

### Importing dataset

In [2]:
df = pd.read_csv('Dataset .csv')
df.head()
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

### Handling the missing values in Dataset

In [3]:
df.fillna(method ='ffill',inplace = True)

C:\Users\Survesh\AppData\Local\Temp\ipykernel_12752\2987202686.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method ='ffill',inplace = True)


### Select relevant features

In [16]:
selected_features =  ['Restaurant Name', 'City', 'Locality', 'Cuisines', 'Average Cost for two', 
                     'Has Table booking', 'Has Online delivery', 'Price range', 
                     'Aggregate rating', 'Votes']
cdf = df[selected_features]
cdf.head()

,Restaurant Name,City,Locality,Cuisines,Average Cost for two,Has Table booking,Has Online delivery,Price range,Aggregate rating,Votes
0,Le Petit Souffle,Makati City,"Century City Mall, Poblacion, Makati City","French, Japanese, Desserts",1100,Yes,No,3,4.8,314
1,Izakaya Kikufuji,Makati City,"Little Tokyo, Legaspi Village, Makati City",Japanese,1200,Yes,No,3,4.5,591
2,Heat - Edsa Shangri-La,Mandaluyong City,"Edsa Shangri-La, Ortigas, Mandaluyong City","Seafood, Asian, Filipino, Indian",4000,Yes,No,4,4.4,270
3,Ooma,Mandaluyong City,"SM Megamall, Ortigas, Mandaluyong City","Japanese, Sushi",1500,No,No,4,4.9,365
4,Sambo Kojin,Mandaluyong City,"SM Megamall, Ortigas, Mandaluyong City","Japanese, Korean",1500,Yes,No,4,4.8,229


### Encode the categorical Variables 

In [17]:
categorical_features =  ['City', 'Locality', 'Cuisines', 'Has Table booking', 'Has Online delivery', 'Price range']
encoder = OneHotEncoder(handle_unknown = 'ignore')
encoded_features = encoder.fit_transform(cdf[categorical_features]).toarray()

###  Merge encoded features back into the dataset

In [18]:
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))
cdf = pd.concat([cdf, encoded_df], axis=1).drop(categorical_features, axis=1)

### Implementing Content Based filtering 

In [19]:
def recommend_restaurants(user_preferences):
    # Convert user preferences to DataFrame with the same feature names
    user_input_df = pd.DataFrame([user_preferences], columns=categorical_features)
    
    # Transform user input using the fitted encoder
    user_vector = encoder.transform(user_input_df).toarray()
    
    # Calculate similarity
    similarity_scores = cosine_similarity(user_vector, encoded_features)
    
    # Recommend top 5 restaurants
    top_indices = np.argsort(similarity_scores[0])[-5:][::-1]
    recommended_restaurants = df.iloc[top_indices][['Restaurant Name', 'Aggregate rating']]
    
    # Sort the recommended restaurants by their aggregate rating in descending order
    recommended_restaurants = recommended_restaurants.sort_values(by='Aggregate rating', ascending=False)

    if len(recommended_restaurants) == 0:
        raise ValueError("No recommendations found for the given preferences.")
    
    # Convert recommendations to a list of dictionaries
    recommendations_list = recommended_restaurants.to_dict(orient='records')
    
    return recommendations_list

# Function to handle the process
def handle_recommendations(user_preferences):
    try:
        recommendations = recommend_restaurants(user_preferences)
        return {
            "status": "success",
            "data": recommendations
        }
    except ValueError as ve:
        return {
            "status": "error",
            "message": str(ve)
        }
    except Exception as e:
        return {
            "status": "error",
            "message": f"An error occurred: {e}"
        }

In [12]:
#Pickeling the Model
import pickle
model_components = {
    'encoder': encoder,
    'encoded_features': encoded_features
}
pickle.dump(model_components,open("Recommendation.pkl","wb"))